<a href="https://colab.research.google.com/github/salmanabbasi36/legal_agent/blob/main/LexGlue_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Colab cell 1: runtime = GPU
!pip install -q transformers datasets accelerate evaluate[sentencepiece] scikit-learn huggingface_hub

In [13]:
from google.colab import drive
drive.mount('/content/drive')  # follow prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from huggingface_hub import login
login()

In [15]:
DATASET = "coastalcph/lex_glue"
DATA_CONFIG = "ledgar"
MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 512
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5
OUTPUT_DIR = '/content/lexglue-ledgar-distilbert'

In [16]:
from datasets import load_dataset
ds = load_dataset(DATASET, DATA_CONFIG)
print(ds)

print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})
{'text': 'Except as otherwise set forth in this Debenture, the Company, for itself and its legal representatives, successors and assigns, expressly waives presentment, protest, demand, notice of dishonor, notice of nonpayment, notice of maturity, notice of protest, presentment for the purpose of accelerating maturity, and diligence in collection.', 'label': 97}


In [17]:
from transformers import AutoTokenizer
import numpy as np

# inspect features to find text field & label field
features = ds['train'].config
print("Feature keys:", features)

#heauristics for text column:

possible_text_cols = ['text', 'sentence', 'promt', 'excerpt', 'passage']
text_col = None
for col in possible_text_cols:
  if col in ds['train'].colmn_name:
    text_col = col
    break

# fall back: choose first string colmn
if text_col is None:
  for k, v in feature.items():
    if str(v).startswith('Value') and k != 'label':
      text_col = k
      break
print("Using label column:", label_col)
# prepare labels
label_feature = features[label_col]
if "ClassLabel" in str(label_feature):
    id2label = label_feature.names
    label2id = {l:i for i,l in enumerate(id2label)}
    num_labels = len(id2label)
    multi_label = False
else:
    # Could be multi-hot or list -> handle as multi-label
    # Convert lists to binary vectors later
    # We'll infer at tokenization time
    id2label = None
    label2id = None
    num_labels = None
    multi_label = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(examples):
    texts = examples[text_col]
    enc = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH)
    # labels
    labs = examples[label_col]
    if not multi_label and id2label is not None:
        enc["labels"] = [label for label in labs]
    else:
        # if labels are lists of ints (multi-label)
        # create binary vector of length K (we can compute K from dataset)
        # compute num_labels if unknown
        global num_labels
        if num_labels is None:
            # estimate max label id +1
            maxid = 0
            for l in ds['train'][label_col]:
                if isinstance(l, list):
                    if l:
                        maxid = max(maxid, max(l))
            num_labels = maxid + 1
        enc["labels"] = [ [1 if i in lab else 0 for i in range(num_labels)] for lab in labs ]
    return enc

tokenized = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)
tokenized = tokenized.shuffle(seed=42)
tokenized



AttributeError: 'Dataset' object has no attribute 'config'

In [18]:
from transformers import AutoTokenizer
import numpy as np

# Inspect features to find text field & label field
features = ds['train'].features
print("Features keys:", features)

# heuristics for text column:
possible_text_cols = ['text', 'sentence', 'prompt', 'excerpt', 'passage']
text_col = None
for col in possible_text_cols:
    if col in ds['train'].column_names:
        text_col = col
        break
# fallback: choose first string column
if text_col is None:
    for k, v in features.items():
        if str(v).startswith('Value') and k != 'label':
            text_col = k
            break

print("Using text column:", text_col)

# label column detection
label_col = None
for c in ['label', 'labels', 'labels_ids', 'gold_label']:
    if c in ds['train'].column_names:
        label_col = c
        break
# fallback: pick first int/class column
if label_col is None:
    for k, v in features.items():
        if "ClassLabel" in str(v) or "Sequence" in str(v):
            label_col = k
            break
print("Using label column:", label_col)

# prepare labels
label_feature = features[label_col]
if "ClassLabel" in str(label_feature):
    id2label = label_feature.names
    label2id = {l:i for i,l in enumerate(id2label)}
    num_labels = len(id2label)
    multi_label = False
else:
    # Could be multi-hot or list -> handle as multi-label
    # Convert lists to binary vectors later
    # We'll infer at tokenization time
    id2label = None
    label2id = None
    num_labels = None
    multi_label = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(examples):
    texts = examples[text_col]
    enc = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH)
    # labels
    labs = examples[label_col]
    if not multi_label and id2label is not None:
        enc["labels"] = [label for label in labs]
    else:
        # if labels are lists of ints (multi-label)
        # create binary vector of length K (we can compute K from dataset)
        # compute num_labels if unknown
        global num_labels
        if num_labels is None:
            # estimate max label id +1
            maxid = 0
            for l in ds['train'][label_col]:
                if isinstance(l, list):
                    if l:
                        maxid = max(maxid, max(l))
            num_labels = maxid + 1
        enc["labels"] = [ [1 if i in lab else 0 for i in range(num_labels)] for lab in labs ]
    return enc

tokenized = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)
tokenized = tokenized.shuffle(seed=42)
tokenized


Features keys: {'text': Value('string'), 'label': ClassLabel(names=['Adjustments', 'Agreements', 'Amendments', 'Anti-Corruption Laws', 'Applicable Laws', 'Approvals', 'Arbitration', 'Assignments', 'Assigns', 'Authority', 'Authorizations', 'Base Salary', 'Benefits', 'Binding Effects', 'Books', 'Brokers', 'Capitalization', 'Change In Control', 'Closings', 'Compliance With Laws', 'Confidentiality', 'Consent To Jurisdiction', 'Consents', 'Construction', 'Cooperation', 'Costs', 'Counterparts', 'Death', 'Defined Terms', 'Definitions', 'Disability', 'Disclosures', 'Duties', 'Effective Dates', 'Effectiveness', 'Employment', 'Enforceability', 'Enforcements', 'Entire Agreements', 'Erisa', 'Existence', 'Expenses', 'Fees', 'Financial Statements', 'Forfeitures', 'Further Assurances', 'General', 'Governing Laws', 'Headings', 'Indemnifications', 'Indemnity', 'Insurances', 'Integration', 'Intellectual Property', 'Interests', 'Interpretations', 'Jurisdictions', 'Liens', 'Litigations', 'Miscellaneous', 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [19]:
from transformers import AutoModelForSequenceClassification
import torch

if not multi_label:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
else:
    # For multi-label classification, set problem_type so Trainer uses BCEWithLogitsLoss
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=num_labels, problem_type="multi_label_classification"
    )

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [20]:
import evaluate
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

metric_acc = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    if multi_label:
        preds = (logits > 0).astype(int)  # logits already converted by trainer to numpy
        # compute micro/macro f1
        return {
            "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
            "f1_macro": f1_score(labels, preds, average="macro", zero_division=0)
        }
    else:
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)
        f1_macro = f1_score(labels, preds, average="macro", zero_division=0)
        return {"accuracy": acc, "f1_macro": f1_macro}


In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro" if not multi_label else "f1_micro",
    fp16=True,  # use mixed precision if supported
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"] if "validation" in tokenized else tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2559876228.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: salmanabbasi36 (salmanabbasi36-beijing-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
metrics = trainer.evaluate()
print(metrics)

# Save locally / to Drive
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved to", OUTPUT_DIR)

# Optionally push to Hub
# trainer.push_to_hub("my-lexglue-ledgar-distilbert")


In [ ]:
from huggingface_hub import login
login()   # it will ask: "Enter your token"


In [ ]:
from huggingface_hub import HfApi
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "SalmanAbbasi/lexglue-ledgar-distilbert"
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)


In [ ]:
from transformers import pipeline
clf = pipeline("text-classification", model=model_name, tokenizer=model_name)
clf("This clause describes the termination of the agreement.")


In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model="/content/lexglue-ledgar-distilbert", tokenizer="/content/lexglue-ledgar-distilbert")

text = "This clause sets out the obligations of each party regarding confidentiality."
print(pipe(text))


In [ ]:
from datasets import load_dataset
ds = load_dataset("coastalcph/lex_glue", "ledgar")
label_names = ds['train'].features['label'].names
print(label_names[:10])  # show first few labels

# Now decode your prediction
result = pipe(text)[0]
label_id = int(result['label'].split('_')[-1])
print(f"Predicted label: {label_names[label_id]} ({result['score']:.2%} confidence)")


In [ ]:
# Update config with id2label and label2id
from transformers import AutoConfig
config = AutoConfig.from_pretrained("/content/lexglue-ledgar-distilbert")
config.id2label = {i: name for i, name in enumerate(label_names)}
config.label2id = {name: i for i, name in enumerate(label_names)}
config.save_pretrained("/content/lexglue-ledgar-distilbert")
